In [1]:
import pandas as pd
import string
import operator

In [3]:
!head -10 tweet_activity_metrics_mozTechSpeakers_20170209_20170511_en.csv

"Tweet id","Tweet permalink","Tweet text","time","impressions","engagements","engagement rate","retweets","replies","likes","user profile clicks","url clicks","hashtag clicks","detail expands","permalink clicks","app opens","app installs","follows","email tweet","dial phone","media views","media engagements","promoted impressions","promoted engagements","promoted engagement rate","promoted retweets","promoted replies","promoted likes","promoted user profile clicks","promoted url clicks","promoted hashtag clicks","promoted detail expands","promoted permalink clicks","promoted app opens","promoted app installs","promoted follows","promoted email tweet","promoted dial phone","promoted media views","promoted media engagements"
"861736323127930881","https://twitter.com/mozTechSpeakers/status/861736323127930881","@trishi_de @lakatos88 @ioana_cis @slsoftworks @manelbutterfly @KohlerSolutions @gabriel_micko We are always happy to see our dinos adopted by nice families. Thank you for coming by!

In [2]:
def load_tweets(tweet_file):

    # Read tweet data 
    tweet_df = pd.read_csv(tweet_file)

    # Drop irrelevant columns
    tweet_df = tweet_df.drop(tweet_df.columns[[13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]], axis=1)

    return tweet_df

In [3]:
tweet_df = load_tweets('tweet_activity_metrics_mozTechSpeakers_20170209_20170511_en.csv')
tweet_df.head()

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,user profile clicks,url clicks,hashtag clicks
0,861736323127930881,https://twitter.com/mozTechSpeakers/status/861...,@trishi_de @lakatos88 @ioana_cis @slsoftworks ...,2017-05-09 00:15 +0000,77.0,3.0,0.038961,0.0,0.0,2.0,0.0,0.0,0.0
1,861735644355276800,https://twitter.com/mozTechSpeakers/status/861...,Thank you for coming by and trying out our dem...,2017-05-09 00:12 +0000,125.0,4.0,0.032000,0.0,0.0,4.0,0.0,0.0,0.0
2,861735332231995392,https://twitter.com/mozTechSpeakers/status/861...,@eDominykas We had a nice booth! Take a look a...,2017-05-09 00:11 +0000,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,861207505153732609,https://twitter.com/mozTechSpeakers/status/861...,@alexnmoldovan @manelbutterfly @ioana_cis @jsh...,2017-05-07 13:14 +0000,80.0,4.0,0.050000,0.0,1.0,0.0,1.0,0.0,0.0
4,861195205621501952,https://twitter.com/mozTechSpeakers/status/861...,The lovely #moztechspeakers working the #mozil...,2017-05-07 12:25 +0000,3080.0,117.0,0.037987,9.0,1.0,36.0,7.0,2.0,3.0


In [4]:
len(tweet_df.index)

33

In [5]:
# Total tweets
print('Total tweets this period:', len(tweet_df.index), '\n')

# Retweets
tweet_df = tweet_df.sort_values(by='retweets', ascending=False)
tweet_df = tweet_df.reset_index(drop=True)
print('Mean retweets:', round(tweet_df['retweets'].mean(),2), '\n') 
print('Top 5 RTed tweets:') 
print('------------------') 
for i in range(5):
    print(tweet_df['Tweet text'].ix[i], '-', tweet_df['retweets'].ix[i]) 
print('\n') 
    
# Likes
tweet_df = tweet_df.sort_values(by='likes', ascending=False)
tweet_df = tweet_df.reset_index(drop=True)
print( 'Mean likes:', round(tweet_df['likes'].mean(),2), '\n')
print('Top 5 liked tweets:') 
print ('-------------------')
for i in range(5):
    print(tweet_df['Tweet text'].ix[i], '-', tweet_df['likes'].ix[i])
print('\n') 

# Impressions
tweet_df = tweet_df.sort_values(by='impressions', ascending=False)
tweet_df = tweet_df.reset_index(drop=True)
print('Mean impressions:', round(tweet_df['impressions'].mean(),2), '\n') 
print('Top 5 tweets with most impressions:')
print('-----------------------------------') 
for i in range(5):
    print(tweet_df['Tweet text'].ix[i], '-', tweet_df['impressions'].ix[i])

Total tweets this period: 33 

Mean retweets: 2.21 

Top 5 RTed tweets:
------------------
Great post by @andgokevin about “Minecraft in WebVR with HTML using @aframevr tutorial available on @Real_CSS_Tricks https://t.co/cgxLrLfyfD - 15.0
📯 Announcing Mobile World Progress Amsterdam, Weds May 10th eve! Feat. @torgo &amp; @ioana_cis. Register now! https://t.co/MjU6jkbCTx  #mwp17 - 13.0
The lovely #moztechspeakers working the #mozilla booth at #jsconfeu. Say 'hi' and ask us anything! https://t.co/KUcAmpKIcC - 9.0
Our very own @AnjanaVakil taking the stage at #jsconfeu, talking immutable #javascript. And we're front and centre to watch the awesomeness https://t.co/X21i35ESy5 - 9.0
We're friendly and we're here to help! 🙌 https://t.co/a6QDXtkN8L - 8.0


Mean likes: 7.79 

Top 5 liked tweets:
-------------------
The lovely #moztechspeakers working the #mozilla booth at #jsconfeu. Say 'hi' and ask us anything! https://t.co/KUcAmpKIcC - 36.0
Our very own @AnjanaVakil taking the stage at #jsco

## TOP hashtags

In [1]:
# Hashtags & mentions
tag_dict = {}
mention_dict = {}

for i in tweet_df.index:
    tweet_text = tweet_df.ix[i]['Tweet text']
    tweet = tweet_text.lower()
    tweet_tokenized = tweet.split()

    for word in tweet_tokenized:
        # Hashtags - tokenize and build dict of tag counts
        if (word[0:1] == '#' and len(word) > 1):
            key = word.translate(str.maketrans("",""))
            if key in tag_dict:
                tag_dict[key] += 1
            else:
                tag_dict[key] = 1

        # Mentions - tokenize and build dict of mention counts
        if (word[0:1] == '@' and len(word) > 1):
            key = word.translate(str.maketrans("",""))
            if key in mention_dict:
                mention_dict[key] += 1
            else:
                mention_dict[key] = 1

# The 10 most popular tags and counts
top_tags = dict(sorted(tag_dict.values(), key=operator.itemgetter(1), reverse=True)[:10])
top_tags_sorted = sorted(top_tags.items(), key=lambda x: x[1])[::-1]
print('Top 10 hashtags:')
print( '----------------')
for tag in top_tags_sorted:
    print( tag[0], '-', str(tag[1]))
    
# The 10 most popular mentions and counts
top_mentions = dict(sorted(mention_dict.iteritems(), key=operator.itemgetter(1), reverse=True))
top_mentions_sorted = sorted(top_mentions.items(), key=lambda x: x[1])[::-1]
print( '\nTop 10 mentions:')
print( '----------------')
# sumln = sum(int(i))
for mention in top_mentions_sorted:
    print( mention[0], '-', str(mention[1]))

NameError: name 'tweet_df' is not defined